In [1]:
import pandas as pd
import numpy as np
import requests

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. 

Make use of the API to find all aggravated burglary incidents that were reported during the six month period from January 1, 2021 through June 30, 2021.

In [25]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.json'

In [39]:
params = {
    'offense_description': 'AGGRAVATED BURGLARY',
    'where': 'reported_date between \'2021-01-01\' AND \'2021-06-30\''
}

In [40]:
response = requests.get(endpoint,
                        params = params )

In [41]:
response

<Response [400]>

In [38]:
response.text

'[]\n'